In [27]:
# import library
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [28]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [29]:
soup = BeautifulSoup(url,'html.parser')
table = soup.find('table',{'class':'wikitable sortable'})
print(table.tr.text)


Postal Code

Borough

Neighborhood



In [30]:
headers = ["PostalCode", "Borough", "Neighborhood1", "Neighborhood2"]

In [31]:
data=""
for tr in table.find_all('tr'):
    row=""
    for tds in tr.find_all('td'):
        row=row+","+tds.get_text()[0:-1]
    data=data+row[1:]+'\n'

In [32]:
file=open("raw.csv","wb")
file.write(bytes(data,encoding="ascii",errors="ignore"))
file.close()

In [33]:
df = pd.read_csv('raw.csv', names=headers)

In [34]:
df.head(10)

,PostalCode,Borough,Neighborhood1,Neighborhood2
0,M1A,Not assigned,Not assigned,NaN
1,M2A,Not assigned,Not assigned,NaN
2,M3A,North York,Parkwoods,NaN
3,M4A,North York,Victoria Village,NaN
4,M5A,Downtown Toronto,Regent Park,Harbourfront
5,M6A,North York,Lawrence Manor,Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park,Ontario Provincial Government
7,M8A,Not assigned,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue,Humber Valley Village
9,M1B,Scarborough,Malvern,Rouge


In [35]:
drop_idx = df[ df['Borough'] =='Not assigned'].index
df.drop(drop_idx , inplace=True)
df.reset_index(drop=True, inplace=True)

In [36]:
nh = pd.Series([]) 
for index, row in df.iterrows():
    if(not pd.isna(row['Neighborhood2'])):
        nh[index] = row['Neighborhood1']+','+row['Neighborhood2']
    else:
        nh[index] = row['Neighborhood1']

In [37]:
df.insert(2, "Neighborhood", nh)
df.drop(['Neighborhood1', 'Neighborhood2'], axis=1, inplace=True)
df.drop_duplicates(subset ="PostalCode", keep = False, inplace = True)

In [38]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [39]:
df.to_csv('CanadaPostalCode.csv', index=False)